In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [19]:
from os import getcwd, chdir

chdir("../")  # Hate this, but quick and dirty

import ast
import logging
import pandas as pd

from codenames.greedy_matrix_helpers import *
from scipy.spatial import distance

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

<IPython.core.display.Javascript object>

# Initial Training

Building on James' work, generate a training matrix for the supervised section of the codenames bot

In [5]:
codenames_df = pd.read_csv("data/5k_conceptnet_game_data/10_dataset.csv")
model_vocab = set(
    open("data/5k_conceptnet_game_data/model_vocab.txt", "r").read().splitlines()
)
model_name = "conceptnet-numberbatch-17-06-300"

model_path = join("models", model_name)
if isfile(model_path):
    model = KeyedVectors.load(model_path)
else:
    model = api.load(model_name)
    model.save(model_path)

restrict_vocab_to_english(model)
restrict_vocab_with_set(model, model_vocab)

<IPython.core.display.Javascript object>

In [6]:
codenames_df

,clue,clue_number,intended_combo,words_guessed,cards,card_types,revealed_before_clue,team,red_team_guesses,blue_team_guesses,assassin_weight,enemy_weight,neutral_weight,ally_weight,risk_weight,clue_score_threshold
0,med,2,"['nurse', 'hospital']","['hospital', 'nurse']","['canada', 'ice', 'egypt', 'contract', 'millio...","['blue', 'blue', 'red', 'red', 'neutral', 'neu...","[False, False, False, True, False, False, Fals...",blue,"['shot', 'contract']","['hospital', 'nurse']",-9.527139,-9.907048,0.513181,5.392982,5.770454,0.373843
1,prosperity,2,"['boom', 'luck']","['luck', 'boom']","['ring', 'whip', 'fire', 'beat', 'plate', 'boo...","['blue', 'neutral', 'red', 'red', 'blue', 'blu...","[False, True, False, False, False, True, True,...",blue,"['ice', 'whip']","['luck', 'boom', 'pan']",-4.348575,-2.490037,-1.444714,4.069033,2.053812,0.350058
2,fingers,2,"['glove', 'hand']","['hand', 'glove']","['bank', 'whale', 'revolution', 'parachute', '...","['blue', 'red', 'assassin', 'neutral', 'red', ...","[True, True, False, False, True, True, True, T...",red,"['hand', 'glove', 'school', 'capital', 'pupil'...","['whale', 'beach', 'antarctica', 'bark', 'lion...",-6.859564,-5.617998,0.452052,4.045837,2.298260,0.250157
3,principal,2,"['pupil', 'capital']","['capital', 'pupil']","['bank', 'whale', 'revolution', 'parachute', '...","['blue', 'red', 'assassin', 'neutral', 'red', ...","[True, True, False, False, True, True, True, T...",red,"['hand', 'glove', 'school', 'capital', 'pupil'...","['whale', 'beach', 'antarctica', 'bark', 'lion...",-6.859564,-5.617998,0.452052,4.045837,2.298260,0.250157
4,cat,2,"['bark', 'lion']","['bark', 'lion']","['bank', 'whale', 'revolution', 'parachute', '...","['blue', 'red', 'assassin', 'neutral', 'red', ...","[True, True, False, False, True, True, True, T...",blue,"['hand', 'glove', 'school', 'capital', 'pupil'...","['whale', 'beach', 'antarctica', 'bark', 'lion...",-6.859564,-5.617998,0.452052,4.045837,2.298260,0.250157
5,shell,1,['slug'],['slug'],"['bank', 'whale', 'revolution', 'parachute', '...","['blue', 'red', 'assassin', 'neutral', 'red', ...","[True, True, False, False, True, True, True, T...",red,"['hand', 'glove', 'school', 'capital', 'pupil'...","['whale', 'beach', 'antarctica', 'bark', 'lion...",-6.859564,-5.617998,0.452052,4.045837,2.298260,0.250157
6,jim,2,"['jack', 'robin']","['jack', 'robin']","['court', 'egypt', 'time', 'contract', 'jack',...","['neutral', 'neutral', 'neutral', 'neutral', '...","[False, False, False, False, True, False, True...",red,"['jack', 'robin', 'china', 'cloak']","['game', 'square', 'horse', 'ring']",-0.189629,-6.891058,-2.619960,6.384826,1.932822,0.282983
7,stadium,2,"['game', 'square']","['game', 'square']","['court', 'egypt', 'time', 'contract', 'jack',...","['neutral', 'neutral', 'neutral', 'neutral', '...","[False, False, False, False, True, False, True...",blue,"['jack', 'robin', 'china', 'cloak']","['game', 'square', 'horse', 'ring']",-0.189629,-6.891058,-2.619960,6.384826,1.932822,0.282983
8,horseshoe,2,"['ring', 'horse']","['horse', 'ring']","['court', 'egypt', 'time', 'contract', 'jack',...","['neutral', 'neutral', 'neutral', 'neutral', '...","[False, False, False, False, True, False, True...",blue,"['jack', 'robin', 'china', 'cloak']","['game', 'square', 'horse', 'ring']",-0.189629,-6.891058,-2.619960,6.384826,1.932822,0.282983
9,los,1,['phoenix'],['phoenix'],"['dress', 'maple', 'penguin', 'soldier', 'clif...","['neutral', 'blue', 'blue', 'red', 'neutral', ...","[False, True, False, True, False, False, True,...",red,"['soldier', 'boot', 'roulette', 'car', 'tokyo'...","['witch', 'missile', 'bridge', 'space', 'hole'...",-4.087929,-0.693982,-5.981360,3.779002,3.121449,0.377077


<IPython.core.display.Javascript object>

In [58]:
def get_repr(self, board):
    x = np.zeros((self.M, self.N), dtype=np.float32)
    for i, word in enumerate(board):
        u = self.kv.get_vector(word)
        for j, v in enumerate(self.v_vocab):
            x[i, j] = 1 - distance.cosine(u, v)

    return x


def get_repr_from_board(board, model):
    x = np.zeros((M, N), dtype=np.float32)
    for i, word in enumerate(board):
        u = model.get_vector(word)
        for j, vocab_word in enumerate(model_vocab_list):
            v = model.get_vector(vocab_word)
            x[i, j] = 1 - distance.cosine(u, v)
    return x

<IPython.core.display.Javascript object>

In [66]:
def gen_samples(n_samples, bg):

    X = np.zeros((n_samples, M, len(bg.tile_types), N))

    board_list = list()
    board_topics_list = list()
    board_types_list = list()

    y_list = list()

    for i in tqdm(range(n_samples)):
        board, board_topics, board_types, y = bg.generate_board()

        # debug
        board_list.append(board)
        board_topics_list.append(board_topics)
        board_types_list.append(board_types)

        x = bg.get_repr(board)

        # TODO efficiency? not actually the slow bit :(
        x_types = [board_type_dict[tt] for tt in board_types]
        for j in range(M):
            card_vocab_dists = x[j]
            ind = x_types[j]
            X[i][j][ind] = card_vocab_dists

        y_list.append(y)

    return X, y_list, board_list, board_topics_list, board_types_list


def gen_samples_from_df(df):

    X = np.zeros((codenames_df.shape[0], M, len(tile_types), N))

    board_list = list()
    board_topics_list = list()
    board_types_list = list()

    y_list = list()

    for i, row in df.iterrows():
        board, board_types, y, y_num, team = (
            ast.literal_eval(row["cards"]),
            ast.literal_eval(row["card_types"]),
            row["clue"],
            row["clue_number"],
            row["team"],
        )
        board_types = [convert_teams_to_enemy_ally(t, team) for t in board_types]

        x = get_repr_from_board(board, model)

        # TODO efficiency? not actually the slow bit :(
        x_types = [board_type_dict[tt] for tt in board_types]
        for j in range(M):
            card_vocab_dists = x[j]
            ind = x_types[j]
            X[i][j][ind] = card_vocab_dists

        y_tuple = (y, model_vocab_list.index(y), y_num)
        y_list.append(y_tuple)

    return X, y_list


def convert_teams_to_enemy_ally(board_type, team):
    enemy_team = "blue" if team == "red" else "red"
    if board_type == team:
        return "ally"
    if board_type == enemy_team:
        return "enemy"
    return board_type

<IPython.core.display.Javascript object>

In [67]:
M = 25
N = len(model_vocab)
model_vocab_list = list(model_vocab)
tile_types = ["ally", "enemy", "neutral", "assassin"]
tile_type_scores = [1, -1, 0, -10]
board_type_dict = {tt: i for i, tt in enumerate(tile_types)}

X, y_list = gen_samples_from_df(codenames_df)


<IPython.core.display.Javascript object>

In [63]:
y_list

['med',
 'prosperity',
 'fingers',
 'principal',
 'cat',
 'shell',
 'jim',
 'stadium',
 'horseshoe',
 'los']

<IPython.core.display.Javascript object>

In [65]:
model_vocab_list.index("med")

940

<IPython.core.display.Javascript object>

In [11]:
for i, row in codenames_df.iterrows():
    print(i)
    print(row["cards"])
    break

0
['canada', 'ice', 'egypt', 'contract', 'millionaire', 'arm', 'snow', 'bill', 'hotel', 'nurse', 'cricket', 'web', 'bark', 'carrot', 'church', 'shot', 'box', 'spider', 'green', 'hospital', 'ray', 'fly', 'palm', 'oil', 'lead']


<IPython.core.display.Javascript object>

In [12]:
codenames_df.head().style

,clue,clue_number,intended_combo,words_guessed,cards,card_types,revealed_before_clue,team,red_team_guesses,blue_team_guesses,assassin_weight,enemy_weight,neutral_weight,ally_weight,risk_weight,clue_score_threshold
0,med,2,"['nurse', 'hospital']","['hospital', 'nurse']","['canada', 'ice', 'egypt', 'contract', 'millionaire', 'arm', 'snow', 'bill', 'hotel', 'nurse', 'cricket', 'web', 'bark', 'carrot', 'church', 'shot', 'box', 'spider', 'green', 'hospital', 'ray', 'fly', 'palm', 'oil', 'lead']","['blue', 'blue', 'red', 'red', 'neutral', 'neutral', 'red', 'neutral', 'blue', 'blue', 'red', 'blue', 'red', 'neutral', 'neutral', 'red', 'red', 'blue', 'neutral', 'blue', 'blue', 'neutral', 'assassin', 'red', 'red']","[False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False]",blue,"['shot', 'contract']","['hospital', 'nurse']",-9.527139,-9.907048,0.513181,5.392982,5.770454,0.373843
1,prosperity,2,"['boom', 'luck']","['luck', 'boom']","['ring', 'whip', 'fire', 'beat', 'plate', 'boom', 'pan', 'telescope', 'code', 'ruler', 'ketchup', 'fork', 'robin', 'comic', 'cap', 'ball', 'sock', 'draft', 'helicopter', 'hospital', 'ice', 'port', 'snow', 'luck', 'bill']","['blue', 'neutral', 'red', 'red', 'blue', 'blue', 'neutral', 'neutral', 'red', 'blue', 'red', 'red', 'neutral', 'red', 'blue', 'blue', 'blue', 'neutral', 'neutral', 'assassin', 'neutral', 'red', 'red', 'blue', 'blue']","[False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False]",blue,"['ice', 'whip']","['luck', 'boom', 'pan']",-4.348575,-2.490037,-1.444714,4.069033,2.053812,0.350058
2,fingers,2,"['glove', 'hand']","['hand', 'glove']","['bank', 'whale', 'revolution', 'parachute', 'india', 'seal', 'beach', 'bark', 'horse', 'pupil', 'note', 'cloak', 'triangle', 'school', 'slug', 'tail', 'glove', 'capital', 'moscow', 'lion', 'ground', 'hand', 'bed', 'time', 'antarctica']","['blue', 'red', 'assassin', 'neutral', 'red', 'blue', 'blue', 'blue', 'red', 'red', 'neutral', 'blue', 'neutral', 'blue', 'red', 'neutral', 'red', 'red', 'neutral', 'blue', 'neutral', 'red', 'red', 'blue', 'neutral']","[True, True, False, False, True, True, True, True, True, True, False, True, False, True, True, False, True, True, False, True, False, True, True, False, True]",red,"['hand', 'glove', 'school', 'capital', 'pupil', 'horse', 'india', 'bed', 'slug']","['whale', 'beach', 'antarctica', 'bark', 'lion', 'seal', 'cloak', 'bank']",-6.859564,-5.617998,0.452052,4.045837,2.298260,0.250157
3,principal,2,"['pupil', 'capital']","['capital', 'pupil']","['bank', 'whale', 'revolution', 'parachute', 'india', 'seal', 'beach', 'bark', 'horse', 'pupil', 'note', 'cloak', 'triangle', 'school', 'slug', 'tail', 'glove', 'capital', 'moscow', 'lion', 'ground', 'hand', 'bed', 'time', 'antarctica']","['blue', 'red', 'assassin', 'neutral', 'red', 'blue', 'blue', 'blue', 'red', 'red', 'neutral', 'blue', 'neutral', 'blue', 'red', 'neutral', 'red', 'red', 'neutral', 'blue', 'neutral', 'red', 'red', 'blue', 'neutral']","[True, True, False, False, True, True, True, True, True, True, False, True, False, True, True, False, True, True, False, True, False, True, True, False, True]",red,"['hand', 'glove', 'school', 'capital', 'pupil', 'horse', 'india', 'bed', 'slug']","['whale', 'beach', 'antarctica', 'bark', 'lion', 'seal', 'cloak', 'bank']",-6.859564,-5.617998,0.452052,4.045837,2.298260,0.250157
4,cat,2,"['bark', 'lion']","['bark', 'lion']","['bank', 'whale', 'revolution', 'parachute', 'india', 'seal', 'beach', 'bark', 'horse', 'pupil', 'note', 'cloak', 'triangle', 'school', 'slug', 'tail', 'glove', 'capital', 'moscow', 'lion', 'ground', 'hand', 'bed', 'time', 'antarctica']","['blue', 'red', 'assassin', 'neutral', 'red', 'blue', 'blue', 'blue', 'red', 'red', 'neutral', 'blue', 'neutral', 'blue', 'red', 'neutral', 'red', 'red', 

<IPython.core.display.Javascript object>